In [5]:
import parse

In [16]:
puzzle.input_data

'depth: 11820\ntarget: 7,782'

In [193]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2018, day=22)

def parses(text):
    depth, target = text.split('\n')
    depth = int(text.split()[1])
    target = parse.search('{:d},{:d}', target).fixed
    return depth, target

data = parses(puzzle.input_data)

In [194]:
sample = parses("""depth: 510
target: 10,10""")

In [51]:
def solve_a(data):
    depth, (W, H) = data
    erosion = np.zeros((W+1,H+1), dtype=np.int32)
    for y in range(H+1):
        for x in range(W+1):
            if y == 0:
                erosion[x,0] = (x*16807 + depth) % 20183
            elif x == 0:
                erosion[0,y] = (y*48271 + depth) % 20183
            else:
                erosion[x,y] = (erosion[x-1,y] * erosion[x,y-1] + depth) % 20183
    erosion[W,H] = erosion[0,0] = 0
    cave = erosion % 3
    return cave.sum()

In [205]:
# from enum import IntEnum
# class Region(IntEnum):
#     Rocky = 0
#     Wet = 1
#     Narrow = 2
        
class Cave:
    
    def __init__(self, depth, target):
        self.depth = depth
        self._erosion = {}
        self.target = target
    
    def __getitem__(self, pos):
        if pos == self.target: # or pos == (0,0):
            return 0
        return self.erosion(*pos) % 3
    
    def erosion(self, x, y):
        if (x,y) not in self._erosion:
            if y == 0:
                gi = x * 16807
            elif x == 0:
                gi = y * 48271
            else:
                gi = self.erosion(x-1,y) * self.erosion(x,y-1)
            self._erosion[x,y] = (gi + self.depth) % 20183
        return self._erosion[x,y]

In [206]:
def solve_a(data):
    depth, (H, W) = data
    cave = Cave(depth, (H, W))
    return sum(cave[i,j] for i in range(H+1) for j in range(W+1))

In [207]:
solve_a(sample)

114

In [208]:
def render(cave):
    W, H = cave.shape
    s = ""
    for y in range(H):
        for x in range(W):
            s += ".=|"[cave[x,y]]
        s += '\n'
    print(s)

In [209]:
solve_a(data)

6318

In [210]:
solve_a(sample)

114

In [211]:
solve_a(data)

6318

In [212]:

    
# class Equip(IntEnum):
#     Neither = 0
#     Torch = 1
#     Climbing = 2
    
# valid_equipment = {
#     Region.Rocky: (Equip.Torch, Equip.Climbing),
#     Region.Wet: (Equip.Neither, Equip.Climbing),
#     Region.Narrow: (Equip.Neither, Equip.Torch),
# }




In [229]:
# from heapq import heappush, heappop
# def solve_b(data):
#     depth, target = data
#     cave = Cave(depth, target)
#     heap = [(0,(0,0), Equip.Torch)]
#     visited = set()
    
#     while True:
#         T, (x,y), equip = heappop(heap)
#         if (x,y) == target and equip == Equip.Torch:
#             return T
#         if (x,y,equip) in visited:
#             continue
#         visited.add((x,y,equip))
        
#         for equip2 in valid_equipment[cave[x,y]]:
#             if (x,y,equip2) not in visited:
#                 heappush(heap, (T+7, (x,y), equip2))
        
#         for x2, y2 in [(x-1,y), (x+1,y), (x,y-1), (x,y+1)]:
#             if (x2 >= 0 and y2 >= 0 and 
#                 equip in valid_equipment[cave[x2,y2]] and
#                 (x2,y2,equip) not in visited):
#                 heappush(heap, (T+1,(x2,y2),equip))
                

In [216]:
# solve_b(data)

In [226]:


from heapq import heappush, heappop
# Note if we let
# rocky=0, wet=1, narrow=2
# neither=0, torch=1, climbing=2
# then equipment is valid iff equip != region
# A* algorithm
def fastsolve_b(data):
    depth, target = data
    X, Y = target
    cave = Cave(depth, target)
    heap = [(X+Y,0,(0,0),1)] 
    visited = set()
    
    while True:
        # Heap of heuristic, time, position, equipment
        C, T, (x,y), equip = heappop(heap)
        if (x,y,equip) in visited:
            continue
        if (x,y) == target and equip == 1:
            return T
        visited.add((x,y,equip))
        
        for equip2 in range(3):
            if equip2 != cave[x,y] and (x,y,equip2) not in visited:
                heappush(heap, (C+7, T+7, (x,y), equip2))
        
        for x2, y2 in [(x-1,y), (x+1,y), (x,y-1), (x,y+1)]:
            if (x2 >= 0 and y2 >= 0 and equip != cave[x2,y2] and
                (x2,y2,equip) not in visited):
                C2 = T+1 + abs(X-x2) + abs(Y-y2)
                heappush(heap, (C2,T+1,(x2,y2),equip))
                

In [227]:
solve_b(sample)

45

In [189]:
# %%timeit
# solve_b(data)

13.3 s ± 337 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [228]:
%%timeit
fastsolve_b(data)

1.92 s ± 67.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
